## Exercise 4: Data Warehouse Querying and Basic Geospatial Operations

Skills: 
* Query data warehouse table
* Use dictionary to map values

References: 
* https://cityoflosangeles.github.io/best-practices/spatial-analysis-basics.html
* https://cityoflosangeles.github.io/best-practices/spatial-analysis-intro.html
* https://cityoflosangeles.github.io/best-practices/spatial-analysis-intermediate.html
* https://cityoflosangeles.github.io/best-practices/data-analysis-intermediate.html
* https://docs.calitp.org/data-infra/analytics_examples/warehouse_tutorial.html
* https://docs.calitp.org/data-infra/analytics_examples/new_tutorial.html
* https://github.com/jorisvandenbossche/geopandas-tutorial

In [ ]:
import geopandas as gpd
import pandas as pd
import os

#os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)
pd.set_option("display.max_rows", 20)

from calitp.tables import tbl
from calitp import query_sql
from siuba import *

## Query a table, turn it into a gdf

For two operators, `ITP_ID == 246` (Caltrain), `ITP_ID == 279` (BART) and `ITP_ID == 343` (Merced the Bus), you will query the warehouse table.

* Query `views.gtfs_schedule_dim_stops`
* Filter to the ITP_ID of interest
* Select these columns: `calitp_itp_id`, `stop_id`, `stop_lat`, `stop_lon`, `stop_name`
* Return as a dataframe using `collect()`
* Turn the point data into geometry with `geopandas`: [docs](https://geopandas.org/en/stable/docs/reference/api/geopandas.points_from_xy.html)

In [ ]:
ITP_ID = [246, 279, 343]
stops = (
    tbl.views.gtfs_schedule_dim_stops()
    >> select(_.itp_id == _.calitp_itp_id, _.stop_id, 
             _.stop_lat, _.stop_lon, _.stop_name)
    >> arrange(_.itp_id, _.stop_id, 
               _.stop_lat, _.stop_lon)
    >> collect()
    >> filter(_.itp_id.isin(ITP_ID))
)

In [ ]:
stops.head()

## Use a dictionary to map values

* Create a new column called `operator` where `itp_id` is associated with its operator name.
* First, write a function to do it.
* Then, use a dictionary to do it (create new column called `agency`).
* Double check that `operator` and `agency` show the same values. Use `assert` to check.
    * `assert df.operator == df.agency`
* Hint: https://cityoflosangeles.github.io/best-practices/data-analysis-intermediate.html

## Turn lat/lon into point geometry

* There is a [function in shared_utils](https://github.com/cal-itp/data-analyses/blob/main/_shared_utils/shared_utils/geography_utils.py#L170-L192) that does it. Show the steps within the function (the long way), and also create the `geometry` column using `shared_utils`.
* Use `geography_utils.create_point_geometry??` to see what goes into that function, and what that function looks like under the hood.

* This GitHub repo has several `geopandas` tutorials that covers basic spatial concepts: https://github.com/jorisvandenbossche/geopandas-tutorial. 
* Skim through the notebooks to see some of the concepts demonstrated, although to actually run the notebooks, you can click on `launch binder` in the repo's README to do so.

## Spatial Join (which points fall into which polygon)

This URL gives you CA county boundaries: https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-county-boundaries/explore?location=37.246136%2C-119.002032%2C6.12

* Go to "I want to use this" > View API Resources > copy link for geojson
* Read in the geojson with `geopandas` and make it a geodataframe: `gpd.read_file(LONG_URL_PATH)`
* Double check that the coordinate reference system is the same for both gdfs using `gdf.crs`. If not, change it so they are the same.
* Spatial join stops to counties: [docs](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html)
    * Play with inner join or left join, what's the difference? Which one do you want?
    * Play with switching around the left_df and right_df, what's the right order?
* By county: count number of stops and stops per sq_mi.
    * Hint 1: Start with a CRS with units in feet or meters, then do a conversion to sq mi. [CRS in shared_utils](https://github.com/cal-itp/data-analyses/blob/main/_shared_utils/shared_utils/geography_utils.py)
    * Hint 2: to find area, you can create a new column and calculate `gdf.geometry.area`. [geometry manipulations docs](https://geopandas.org/en/stable/docs/user_guide/geometric_manipulations.html)